# Data Processing and Splitting

In this notebook, with the steps below:


1. **Removing Unnecessary Features**: We clean the data by removing features that are redundant, irrelevant, or may negatively impact downstream analysis.

2. **Batch Correction and Normalization**: Since we are working with an aggregated dataset, we apply batch correction using spherization, leveraging the Pycytominer library to account for batch effects and ensure that the data is standardized for further analysis.

3. **Well Holdouts**: We generate well holdouts by randomly selecting specific wells to form our holdout dataset. This ensures that the model's performance is evaluated at the well level, helping to assess its generalizability across different wells.

4. **Data Splitting**: We then split the data into training and testing sets to effectively train the model while preserving a separate set for performance evaluation.

In [1]:
import json
import pathlib
from typing import Optional, Tuple
import numpy as np
import pandas as pd
from pycytominer import normalize
from pycytominer.cyto_utils import infer_cp_features
from sklearn.model_selection import train_test_split


## Helper functions

In [2]:
def split_meta_and_features(
    profile: pd.DataFrame,
    compartments=["Nuclei", "Cells", "Cytoplasm"],
    metadata_tag: Optional[bool] = False,
) -> Tuple[list[str], list[str]]:
    """Splits metadata and feature column names

    Parameters
    ----------
    profile : pd.DataFrame
        image-based profile
    compartments : list, optional
        compartments used to generated image-based profiles, by default
        ["Nuclei", "Cells", "Cytoplasm"]
    metadata_tag : Optional[bool], optional
        indicating if the profiles have metadata columns tagged with 'Metadata_'
        , by default False

    Returns
    -------
    tuple[list[str], list[str]]
        Tuple containing metadata and feature column names
    """

    # identify features names
    features_cols = infer_cp_features(profile, compartments=compartments)

    # iteratively search metadata features and retain order if the Metadata tag is not added
    if metadata_tag is False:
        meta_cols = [
            colname
            for colname in profile.columns.tolist()
            if colname not in features_cols
        ]
    else:
        meta_cols = infer_cp_features(profile, metadata=metadata_tag)

    return (meta_cols, features_cols)

In [3]:
# global varaibles
seed = 0

# path to the data
data_path = pathlib.Path("../data/labeled_cell_injury.csv")
json_path = pathlib.Path("../data/cell_injury_shared_feature_space.json")

In [4]:
# loading json feature spce
with open(json_path, mode="r") as infile:
    features = json.load(infile)

# loading jumop_data
jump_df = pd.read_csv(data_path)
jump_df.head()

# separating mdetadata and feature column names
jump_meta, jump_feats = split_meta_and_features(profile=jump_df)

/tmp/ipykernel_2139869/3926556349.py:6: DtypeWarning: Columns (12,13,24) have mixed types. Specify dtype option on import or set low_memory=False.
  jump_df = pd.read_csv(data_path)


## Removing unnecessary features

In [5]:
# features to remove
featured_to_remove = [
    "Cytoplasm_AreaShape_Center_X",
    "Cells_AreaShape_Center_Y",
    "Cells_AreaShape_Orientation",
    "Nuclei_AreaShape_Orientation",
    "Cells_AreaShape_Orientation",
    "Cytoplasm_AreaShape_Extent",
]

# removing unnecessary morphological features
filtered_feats = [feat_name for feat_name in features["features"] if feat_name not in featured_to_remove]

In [6]:
# update the jump dataset
jump_df = jump_df[jump_meta + filtered_feats].rename(columns={"Control Type": "control_type"}).head()
jump_meta, jump_feats = split_meta_and_features(jump_df)

In [7]:
jump_df[jump_meta]

,injury_type,Plate,Well,Characteristics [Organism],Term Source 1 REF,Term Source 1 Accession,Characteristics [Cell Line],Term Source 2 REF,Term Source 2 Accession,Experimental Condition [Treatment time (h)],...,Compound PubChem CID,Compound PubChem URL,control_type,Channels,Comment [Image File Path],Comment [Image Prefix],Mahalanobis distance,Mahalanobis distance significant,Relative well cellcount,Relative well cellcount significant
0,Control,BR00110363,B2,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,679.0,https://pubchem.ncbi.nlm.nih.gov/compound/679,Negative,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",/incoming/BR00110363/,r02c02,7.51,No,1.02,No
1,Control,BR00110363,B3,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,679.0,https://pubchem.ncbi.nlm.nih.gov/compound/679,Negative,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",/incoming/BR00110363/,r02c03,6.21,No,1.11,No
2,Control,BR00110363,B4,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,679.0,https://pubchem.ncbi.nlm.nih.gov/compound/679,Negative,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",/incoming/BR00110363/,r02c04,10.94,No,1.02,No
3,Control,BR00110363,B5,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,679.0,https://pubchem.ncbi.nlm.nih.gov/compound/679,Negative,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",/incoming/BR00110363/,r02c05,7.59,No,1.06,No
4,Control,BR00110363,B6,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,679.0,https://pubchem.ncbi.nlm.nih.gov/compound/679,Negative,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",/incoming/BR00110363/,r02c06,5.28,No,1.00,No


## Adding injury labels to dataset

## Batch correction 

In [8]:
# applying whiting to correct for batch effects found within the both
norm_df = normalize(profiles=jump_df,
                      meta_features=jump_meta,
                      features=filtered_feats,
                      samples="control_type == 'Negative'",
                      method="spherize",
                      spherize_center=True,
                      spherize_method="ZCA-cor",
                      spherize_epsilon=1e-6)

# display
norm_df.head()

,injury_type,Plate,Well,Characteristics [Organism],Term Source 1 REF,Term Source 1 Accession,Characteristics [Cell Line],Term Source 2 REF,Term Source 2 Accession,Experimental Condition [Treatment time (h)],...,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_RadialDistribution_MeanFrac_DNA_2of4,Nuclei_Intensity_StdIntensityEdge_DNA,Nuclei_Intensity_StdIntensityEdge_ER,Cells_AreaShape_Perimeter,Cytoplasm_Intensity_MADIntensity_DNA,Cells_RadialDistribution_RadialCV_Mito_2of4,Cells_RadialDistribution_MeanFrac_ER_3of4,Cytoplasm_AreaShape_Zernike_1_1,Nuclei_Intensity_MinIntensity_ER
0,Control,BR00110363,B2,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,-0.204854,0.051142,-0.040272,-0.056018,0.106578,-0.065333,0.121630,0.038548,-0.155018,-0.141351
1,Control,BR00110363,B3,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.076026,0.096445,0.014056,0.194627,-0.027585,-0.055004,0.013849,-0.114215,-0.056530,0.048632
2,Control,BR00110363,B4,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.054373,0.095926,-0.193571,-0.193723,0.034240,-0.157140,-0.210162,-0.094891,0.101525,-0.007450
3,Control,BR00110363,B5,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,-0.045621,-0.251438,0.147624,0.005140,-0.222666,0.148643,0.059836,-0.045647,0.207362,0.198444
4,Control,BR00110363,B6,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,24,...,0.120076,0.007925,0.072164,0.049973,0.109433,0.128833,0.014847,0.216205,-0.097339,-0.098276


## Generating Holdouts

### Well level holdouts

In [9]:
# numober of wells to select per plate
n_controls = 5
n_samples = 10

# setting random seed globally
np.random.seed(seed)

# collecting randomly select wells based on treatment
wells_heldout_df = []
for plate, df in norm_df.groupby("Plate", as_index=False):

    # separate control wells and rest of all wells since there is a huge label imbalance
    # selected 5 control wells and 10 random wells from the plate
    df_control = df.loc[df["control_type"] == "Negative"].sample(
        n=n_controls, random_state=seed
    )
    df_treated = df.loc[df["control_type"] != "Negative"].sample(
        n=n_samples, random_state=seed
    )


    # concatenate those together
    well_heldout = pd.concat([df_control, df_treated])

    wells_heldout_df.append(well_heldout)

# generate well holdout dataframe
wells_heldout_df = pd.concat(wells_heldout_df)

# take the indices of the held out data frame and use it to drop those samples from
# the main dataset. And then check if those indices are dropped
wells_idx_to_drop = wells_heldout_df.index.tolist()
norm_df = norm_df.drop(wells_idx_to_drop)
assert all(
    [
        True if num not in norm_df.index.tolist() else False
        for num in wells_idx_to_drop
    ]
), "index to be dropped found in the main dataframe"

# saving the holdout data
wells_heldout_df.to_csv(
     "./wells_holdout.csv.gz", index=False, compression="gzip"
)

# display
print("Wells holdout shape:", wells_heldout_df.shape)
wells_heldout_df.head()

ValueError: a must be greater than 0 unless no samples are taken

## Splitting the data 

In [ ]:
# split the data into trianing and testing sets
meta_cols, _ = split_meta_and_features(norm_df)
X = norm_df[fs_feats]
y = norm_df["injury_code"]

# splitting dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.80, random_state=seed, stratify=y
)

# saving training dataset as csv file
data_dir = pathlib.Path("../data").resolve(strict=True)
X_train.to_csv(data_dir / "X_train.csv.gz", compression="gzip", index=False)
X_test.to_csv(data_dir / "X_test.csv.gz", compression="gzip", index=False)
y_train.to_csv(data_dir / "y_train.csv.gz", compression="gzip", index=False)
y_test.to_csv(data_dir / "y_test.csv.gz", compression="gzip", index=False)

# display data split sizes
print("X training size", X_train.shape)
print("X testing size", X_test.shape)
print("y training size", y_train.shape)
print("y testing size", y_test.shape)